In [2]:
import pandas as pd
import numpy as np
import math
# Method 1: Linear transformation.

#path: the path of panel (data missing is allowed)
panel = pd.read_excel("path")  
# A custom substring function for seeking reference interval location
def isSubstring(s1,s2,s):                            
    tag = False
    if s.find(s1) != -1 or s.find(s2) != -1: 
        tag = True
    return tag
# Range_loc: Store reference interval location.
Range_loc=[]                                         
name=panel.columns.values
str1=str('lower')
str2=str('upper')
for i in range(len(name)):
    if isSubstring(str1,str2,name[i])== True:
        Range_loc.append(i)
    

# number_of_items: the number of test item in the panel.
number_of_items=int(len(Range_loc)/2) 
# Keep only the test items that need to be transformed and the corresponding reference interval
panel = panel.iloc[:,2:(2+3*number_of_items)]
table = panel.iloc[:,number_of_items:3*number_of_items].value_counts()
panel_lineartrans = panel.copy(deep=True).astype('float')  
standard=panel.iloc[:,number_of_items:3*number_of_items].value_counts().index[0]
for i in range(number_of_items):
    transformation_value =  (panel_lineartrans.iloc[:,i]-panel_lineartrans.iloc[:,2*i+number_of_items])*(standard[2*i+1]-standard[2*i])/(panel_lineartrans.iloc[:,2*i+number_of_items+1]-panel_lineartrans.iloc[:,2*i+number_of_items])+standard[2*i] 
    panel_lineartrans.iloc[:,i] = [x if x > 0 else 0 for x in transformation_value]       
    # Change the reference interval used.
    panel_lineartrans.iloc[:,2*i+number_of_items] = standard[2*i]
    panel_lineartrans.iloc[:,2*i+number_of_items+1] = standard[2*i+1]
    
df = pd.read_excel("path")  
missing_cols = [c for c in df.columns if c not in panel_lineartrans.columns]

# 2) 将缺失列从 df1 补到 df2
df2 = pd.concat([panel_lineartrans, df[missing_cols]], axis=1)

# 3) 按照 df1 的列顺序重排（只保留 df1 中的列）
df2 = df2[df.columns]
panel_lineartrans = df2

#linear-transformated result
panel_lineartrans

,sex,age,prothrombin time,activated partial thromboplastin time,fibrinogen,thrombin time,prothrombin time_lower,prothrombin time_upper,activated partial thromboplastin time_lower,activated partial thromboplastin time_upper,fibrinogen_lower,fibrinogen_upper,thrombin time_lower,thrombin time_upper,hospital
0,0,78,12.600000,35.3,3.180000,19.400000,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
1,1,86,14.441667,31.9,4.904348,12.542857,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
2,0,84,12.800000,38.1,3.310000,17.600000,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
3,1,69,14.075000,37.9,3.660870,16.657143,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
4,1,67,13.800000,41.7,5.080000,21.100000,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,79,12.516667,26.7,2.008696,18.714286,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
496,0,57,13.891667,38.6,4.739130,17.171429,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
497,0,71,14.100000,43.0,5.130769,16.310000,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
498,0,37,12.600000,39.4,2.453846,17.480000,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1


In [3]:
# Method 2: ECDF transformation

# path: the path of panel 
panel = pd.read_excel("path")  

# A custom substring function for seeking reference interval location
def isSubstring(s1,s2,s):                            
    tag = False
    if s.find(s1) != -1 or s.find(s2) != -1: 
        tag = True
    return tag
# Range_loc: Store reference interval location.
Range_loc=[]                                         
name=panel.columns.values
str1=str('lower')
str2=str('upper')
for i in range(len(name)):
    if isSubstring(str1,str2,name[i])== True:
        Range_loc.append(i)
    
# number_of_items: the number of test item in the panel.
number_of_items=int(len(Range_loc)/2) 
# Keep only the test items that need to be transformed and the corresponding reference interval
panel = panel.iloc[:,2:(2+3*number_of_items)]
table = panel.iloc[:,number_of_items:3*number_of_items].value_counts()
panel_ecdf = panel.copy(deep=True).astype('float')  
threshold_set = 30
Number_of_ranges_com = np.sum(table.values>threshold_set)

index=[[]for z in range(Number_of_ranges_com)]
key=0
for z in range(Number_of_ranges_com):
    index[z]=np.where(np.sum(panel.iloc[:,number_of_items:3*number_of_items]==table.index[z],axis=1)==2*number_of_items)[0].tolist()
for i in range(number_of_items):
    n=[[]for z in range(Number_of_ranges_com)]                   #The number of occurrences of each reference interval
    n1=[[]for z in range(Number_of_ranges_com)]                  #The number of occurrences below the lower bound in each reference interval
    n2=[[]for z in range(Number_of_ranges_com)]                  #The number of occurrences above the upper bound in each reference interval
    F=[[]for z in range(Number_of_ranges_com)]                   #Quantile obtained by maximum likelihood
    P1=[[]for z in range(Number_of_ranges_com)]                  #The proportion of the test conclusion below the normal value under different reference interval
    P2=[[]for z in range(Number_of_ranges_com)]   
    for z in range(Number_of_ranges_com):
        n[z]=table.values[z]
        n1[z]=np.sum(panel.iloc[index[z],i]<=table.index[z][2*i])
        n2[z]=np.sum(panel.iloc[index[z],i]>table.index[z][2*i+1])
        P1[z]=np.sum(panel.iloc[index[z],i]<=table.index[z][2*i])/n[z]
        P2[z]=np.sum(panel.iloc[index[z],i]>table.index[z][2*i+1])/n[z]
        #panel_ecdf.iloc[index[z],i]=(panel_ecdf.iloc[index[z],i]-table.index[z][2*i])*(table.index[0][2*i+1]-table.index[0][2*i])/(table.index[z][2*i+1]-table.index[z][2*i])+table.index[0][2*i]
    if(all(P1) or not any(P1))and (all(P2) or not any(P2)): 
        x=np.sum(n1)/np.sum(n)
        y=np.sum(n2)/np.sum(n)
        for j in range(Number_of_ranges_com):
            rank=panel.iloc[index[j],i].rank().tolist()      #Ranking of observations under j-th normal range
            F[j]=[[0]for u in range(n[j])]                   #Empirical cumulative density
            for k in range(n[j]):
                if rank[k]<=n1[j]:
                    F[j][k]=rank[k]*x/n1[j]
                if rank[k]>n1[j] and rank[k]<=n[j]-n2[j]:
                    F[j][k]=(rank[k]-n1[j])*(1-x-y)/(n[j]-n1[j]-n2[j])+x
                if rank[k]>n[j]-n2[j]:
                    F[j][k]=(rank[k]-(n[j]-n2[j]))*(y/n2[j])+1-y
        for j in range(Number_of_ranges_com):
            if j!=key:
                for k in range(n[j]):
                    panel_ecdf.iloc[index[j][k],i]=panel_ecdf.iloc[index[key][np.argmin(abs(F[j][k]-F[key]))],i]
# Change the reference interval used.
panel_ecdf.iloc[:,number_of_items:3*number_of_items] = table.index[0]

df = pd.read_excel("path")  
missing_cols = [c for c in df.columns if c not in panel_ecdf.columns]

# 2) 将缺失列从 df1 补到 df2
df2 = pd.concat([panel_ecdf, df[missing_cols]], axis=1)

# 3) 按照 df1 的列顺序重排（只保留 df1 中的列）
df2 = df2[df.columns]
panel_ecdf = df2
#ECDF-transformated result
panel_ecdf

,sex,age,prothrombin time,activated partial thromboplastin time,fibrinogen,thrombin time,prothrombin time_lower,prothrombin time_upper,activated partial thromboplastin time_lower,activated partial thromboplastin time_upper,fibrinogen_lower,fibrinogen_upper,thrombin time_lower,thrombin time_upper,hospital
0,0,78,12.6,35.3,3.18,19.4,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
1,1,86,13.8,32.1,5.65,15.2,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
2,0,84,12.8,38.1,3.31,17.6,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
3,1,69,13.5,37.3,3.84,18.4,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
4,1,67,13.8,41.7,5.08,21.1,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,79,12.2,30.0,2.14,20.0,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
496,0,57,13.3,38.3,5.34,18.8,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,2
497,0,71,13.9,42.6,5.78,16.9,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
498,0,37,12.5,39.2,2.77,18.2,10.5,16.0,30.0,50.0,2.0,4.0,11.0,20.0,1
